In [2]:
!pip install Flask

In [1]:
from flask import Flask, render_template, request, jsonify, send_from_directory
from werkzeug.utils import secure_filename
import os
import cv2
import torch
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer, ColorMode

# Create the Flask app
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['MODEL_PATH'] = r"C:\Users\Riria\OneDrive\Desktop\AI project\model_0004999.pth"  # Updated model path

os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# Load the model
def load_model():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.MODEL.WEIGHTS = app.config['MODEL_PATH']
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    predictor = DefaultPredictor(cfg)
    return predictor

predictor = load_model()

# Prediction function
def predict_mask_rcnn(image_path, predictor):
    image = cv2.imread(image_path)
    outputs = predictor(image)
    scores = outputs["instances"].scores.cpu().numpy()
    max_score_index = scores.argmax()
    top_score = scores[max_score_index] * 100
    predicted_label = "Mitotic Figure" if len(scores) > 0 else "No Mitotic Figure Detected"
    v = Visualizer(image[:, :, ::-1], metadata=None, scale=0.5, instance_mode=ColorMode.IMAGE)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    result_image = v.get_image()[:, :, ::-1]
    return result_image, predicted_label, round(top_score, 2)

# Routes for Flask
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['file']
    filename = secure_filename(file.filename)
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(filepath)
    segmented_image, prediction, confidence = predict_mask_rcnn(filepath, predictor)
    output_filename = f"segmented_{filename}"
    output_path = os.path.join(app.config['UPLOAD_FOLDER'], output_filename)
    cv2.imwrite(output_path, segmented_image)
    return jsonify({'image_url': f'uploads/{output_filename}', 'prediction': prediction, 'confidence': confidence})

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'detectron2'